Coneccion bd

In [1]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

# Credenciales db
host = 'localhost'
database = 'DB_CODELCO_DRT'
username = 'postgres'  
password = '1234' 

# Crea la coneccion
conn = psycopg2.connect(
    host=host,
    database=database,
    user=username,
    password=password
)

engine = create_engine('postgresql://'+username+':'+password+'@'+host+'/'+database)
table_name1 = 'HG_DRT_Infraestructuras'
table_name2 = 'HG_DRT_Niveles'
table_name3 = 'Act_niveles_26_06_24'


df_infraestructuras_bd = pd.read_sql_table(table_name1, con=engine)#Trae la tabla de la bda un dataframe
df_niveles_bd = pd.read_sql_table(table_name2, con=engine)#
Nivelesupdate = pd.read_sql_table(table_name3, con=engine)#

In [2]:
df_niveles_bd.columns

Index(['Object_ID', 'HoleID', 'Date', 'Hora', 'Depth', 'dry_indicator',
       'StickUp', 'Nivel_Estatico_(msnm)', 'Nivel_Estatico_Calculado_(msnm)',
       'Personal', 'comment', 'Source', 'Data_Source'],
      dtype='object')

LEER ARCHIVO NUEVA DATA

In [9]:
data_new = pd.read_excel(r'G:\Unidades compartidas\03.3 Proyectos Codelco-Dist. Norte\COD006-OTM N° 2 DRT Mod. Transporte\03.WIP\01.Hidrogeología\Niveles\Trabajo\Niveles_final_MA.xlsx', sheet_name='Datos', header=0)

In [10]:
data_new.columns

Index(['HoleID', 'Sector', 'E_WSG-84', 'N_WSG-84',
       'Elevación_Instrumento (msnm) (con TOC)', 'StickUp', 'Date', 'Hora',
       'Depth', 'Nivel_Estatico_(msnm)', 'Nivel_Estatico_Calculado_(msnm)',
       'comment', 'Source', 'Data_Source'],
      dtype='object')

EXTRAEMOS LAS COLUMNAS NECESARIAS

In [11]:
extract_columns = ['HoleID', 'Date', 'Hora', 'Sector',
       'StickUp', 'Nivel_Estatico_(msnm)', 'Nivel_Estatico_Calculado_(msnm)','Elevación_Instrumento (msnm) (con TOC)',
       'comment',]
data_new = data_new[extract_columns]

In [12]:
data_new

,HoleID,Date,Hora,Sector,StickUp,Nivel_Estatico_(msnm),Nivel_Estatico_Calculado_(msnm),Elevación_Instrumento (msnm) (con TOC),comment
0,AR-1569,2024-03-21 00:00:00,11:50:00,Pampa Cere,NaN,2644.059,2644.059,2656.29,NaN
1,AR-1569,2024-01-22 00:00:00,10:05:00,Pampa Cere,NaN,2644.065,2644.065,2656.29,NaN
2,AR-1569,2023-12-21 00:00:00,10:50:00,Pampa Cere,NaN,2643.996,2643.996,2656.29,NaN
3,AR-1569,2023-11-24 00:00:00,14:50:00,Pampa Cere,NaN,2643.934,2643.934,2656.29,NaN
4,AR-1569,2023-10-20 00:00:00,09:56:00,Pampa Cere,NaN,2644.065,2644.065,2656.29,NaN
...,...,...,...,...,...,...,...,...,...
557318,RH-08,2019-08-27 03:00:00,NaN,Rajo RT,-,2843.279906,NaN,3002.3,NaN
557319,RH-08,2019-08-27 15:00:00,NaN,Rajo RT,-,2843.279906,NaN,3002.3,NaN
557320,RH-08,2019-08-28 15:00:00,NaN,Rajo RT,-,2843.268001,NaN,3002.3,NaN
557321,RH-08,2019-09-20 15:00:00,NaN,Rajo RT,-,2842.65,NaN,3002.3,NaN


SE GENERA JOIN DE LA NUEVA INFO CON LA DATA INFRAESTRUCTURA EN LA BD PARA DAR FORMATO

In [20]:
# df_combined = pd.merge(data_new, df_infraestructuras_bd, 
#                        left_on='HoleID', 
#                        right_on='HoleID', 
#                        how='left')

df_combined = pd.merge(
       data_new[
       ['HoleID', 'Date', 'Hora', 'Sector',
       'StickUp', 'Nivel_Estatico_(msnm)', 'Nivel_Estatico_Calculado_(msnm)','Elevación_Instrumento (msnm) (con TOC)',
       'comment']  #Columnas que traemos de la tabla base
       ], 
       df_infraestructuras_bd[
              ['HoleID','Object_ID','Depth','Source', 'Data_Source'] #Columnas que traemos de la tabla infraestructura
              ], 
       on='HoleID', how='inner')

In [15]:
df_combined

,HoleID,Date,Hora,Sector,StickUp,Nivel_Estatico_(msnm),Nivel_Estatico_Calculado_(msnm),Elevación_Instrumento (msnm) (con TOC),comment,Object_ID,Depth,Source,Data_Source
0,AR-1569,2024-03-21 00:00:00,11:50:00,Pampa Cere,NaN,2644.059,2644.059,2656.29,NaN,6,NaN,Pozos con Estratigrafía_proyectados.xls,\\cl0014inf01\22-Projects\026_Radomiro_Tomic\5...
1,AR-1569,2024-01-22 00:00:00,10:05:00,Pampa Cere,NaN,2644.065,2644.065,2656.29,NaN,6,NaN,Pozos con Estratigrafía_proyectados.xls,\\cl0014inf01\22-Projects\026_Radomiro_Tomic\5...
2,AR-1569,2023-12-21 00:00:00,10:50:00,Pampa Cere,NaN,2643.996,2643.996,2656.29,NaN,6,NaN,Pozos con Estratigrafía_proyectados.xls,\\cl0014inf01\22-Projects\026_Radomiro_Tomic\5...
3,AR-1569,2023-11-24 00:00:00,14:50:00,Pampa Cere,NaN,2643.934,2643.934,2656.29,NaN,6,NaN,Pozos con Estratigrafía_proyectados.xls,\\cl0014inf01\22-Projects\026_Radomiro_Tomic\5...
4,AR-1569,2023-10-20 00:00:00,09:56:00,Pampa Cere,NaN,2644.065,2644.065,2656.29,NaN,6,NaN,Pozos con Estratigrafía_proyectados.xls,\\cl0014inf01\22-Projects\026_Radomiro_Tomic\5...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
557318,RH-08,2019-08-27 03:00:00,NaN,Rajo RT,-,2843.279906,NaN,3002.3,NaN,272,220.0,Catastro_01_06_2011.xlsx,\\cl0014inf01\22-Projects\026_Radomiro_Tomic\5...
557319,RH-08,2019-08-27 15:00:00,NaN,Rajo RT,-,2843.279906,NaN,3002.3,NaN,272,220.0,Catastro_01_06_2011.xlsx,\\cl0014inf01\22-Projects\026_Radomiro_Tomic\5...
557320,RH-08,2019-08-28 15:00:00,NaN,Rajo RT,-,2843.268001,NaN,3002.3,NaN,272,220.0,Catastro_01_06_2011.xlsx,\\cl0014inf01\22-Projects\026_Radomiro_Tomic\5...
557321,RH-08,2019-09-20 15:00:00,NaN,Rajo RT,-,2842.65,NaN,3002.3,NaN,272,220.0,Catastro_01_06_2011.xlsx,\\cl0014inf01\22-Projects\026_Radomiro_Tomic\5...


EXTRAEMOS SOLO LAS COLUMNAS NECESARIA PARA LA TABLA NIVELES

In [16]:
columns_extract = ['Object_ID', 'HoleID', 'Sector' ,'Date', 'Hora', 'Depth',
       'StickUp', 'Nivel_Estatico_(msnm)', 'Nivel_Estatico_Calculado_(msnm)',
        'comment', 'Source', 'Data_Source']

df_combined = df_combined[columns_extract]

CONVERTIR COLUMNAS A CAMPO NUMERICO

In [18]:

# transformar una columna a float en un dataframe y que los valores qye no se puedan transformar se reemplacen por vacio
df_combined['Nivel_Estatico_(msnm)'] = pd.to_numeric(df_combined['Nivel_Estatico_(msnm)'], errors='coerce')

df_combined['Nivel_Estatico_Calculado_(msnm)'] = pd.to_numeric(df_combined['Nivel_Estatico_Calculado_(msnm)'], errors='coerce')

C:\Users\fdona\AppData\Local\Temp\ipykernel_1772\2194811949.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined['Nivel_Estatico_(msnm)'] = pd.to_numeric(df_combined['Nivel_Estatico_(msnm)'], errors='coerce')
C:\Users\fdona\AppData\Local\Temp\ipykernel_1772\2194811949.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined['Nivel_Estatico_Calculado_(msnm)'] = pd.to_numeric(df_combined['Nivel_Estatico_Calculado_(msnm)'], errors='coerce')


SUBIMOS LA TABLA A LA BD CORRESPONDIENTE

In [19]:
df_combined.to_sql('Act_niveles_26_06_24', con=engine, if_exists='replace', index=False)

323

In [44]:
df_combined.to_excel(r'G:\Unidades compartidas\03. Proyectos\20. FLOW\FLOW-003 Sistema Gestión de la Información\10_Colaborativos\01_Solicitudes\CODELCO-Dist. Norte\Modelo_Transporte_DRT\Base_de_datos_DRT_Mod_Trans_26062024\BD-DRT-NIVELES_26062024.xlsx', sheet_name='Datos', index=False)

In [45]:
df_infraestructuras_bd.to_excel(r'G:\Unidades compartidas\03. Proyectos\20. FLOW\FLOW-003 Sistema Gestión de la Información\10_Colaborativos\01_Solicitudes\CODELCO-Dist. Norte\Modelo_Transporte_DRT\Base_de_datos_DRT_Mod_Trans_26062024\BD-DRT-INFRAESTRUCTURAS_26062024.xlsx', sheet_name='Datos', index=False)

CALCULAR TEMPORALIDAD

In [48]:
import pandas as pd

def generar_temporalidad(archivo_csv):
    """
    Genera un DataFrame de temporalidad con la fecha mínima, la fecha máxima y la cantidad de datos
    por 'Nombre infraestructura' a partir de un dataset de concentraciones.

    Returns:
    pd.DataFrame: DataFrame con la temporalidad
    """
    # Cargar el archivo CSV en un DataFrame
    df = archivo_csv
    
    # Convertir la columna 'Fecha' a tipo datetime
    # df['Fecha'] = pd.to_datetime(df['FechaMuestra'])
    
    # Agrupar por 'Nombre infraestructura' y calcular fecha mínima, fecha máxima y cantidad de datos
    temporalidad_df = df.groupby('HoleID').agg(
        fecha_min=('Date', 'min'),
        fecha_max=('Date', 'max'),
        cantidad_Muestras_Unicas_ne=('Nivel_Estatico_(msnm)', 'count'),
        cantidad_Muestras_Unicas_nc=('Nivel_Estatico_Calculado_(msnm)', 'count')
    ).reset_index()
    # temporalidad_df['fecha_min'] = temporalidad_df['fecha_min'].dt.strftime('%d %B %Y')
    # temporalidad_df['fecha_max'] = temporalidad_df['fecha_max'].dt.strftime('%d %B %Y')
    
    return temporalidad_df

# Uso de la función
archivo = 'ruta/al/archivo.csv'  # Reemplaza con la ruta a tu archivo CSV
resultado = generar_temporalidad(df_niveles_bd)
print(resultado)
resultado

      HoleID           fecha_min           fecha_max  \
0    AR-1569 2003-10-24 00:00:00 2024-03-21 00:00:00   
1    AR-4705 2002-12-18 00:00:00 2007-10-17 00:00:00   
2    AR-4712 2002-12-18 00:00:00 2006-10-16 00:00:00   
3    AR-4714 2002-12-18 00:00:00 2007-09-12 00:00:00   
4    AR-4739 2002-12-18 00:00:00 2008-06-12 00:00:00   
..       ...                 ...                 ...   
203   TL-09C 2005-05-10 15:45:00 2005-11-26 12:10:00   
204    TL-14 2005-05-10 16:40:00 2005-11-08 15:45:00   
205   TL-34C 2005-05-11 12:30:00 2005-11-26 14:10:00   
206   TL-35C 2005-02-06 00:00:00 2024-01-30 00:00:00   
207    TT-4E 2005-05-08 10:39:00 2005-11-26 14:35:00   

     cantidad_Muestras_Unicas_ne  cantidad_Muestras_Unicas_nc  
0                            262                           89  
1                             36                            0  
2                             41                            9  
3                             37                            4  
4      

,HoleID,fecha_min,fecha_max,cantidad_Muestras_Unicas_ne,cantidad_Muestras_Unicas_nc
0,AR-1569,2003-10-24 00:00:00,2024-03-21 00:00:00,262,89
1,AR-4705,2002-12-18 00:00:00,2007-10-17 00:00:00,36,0
2,AR-4712,2002-12-18 00:00:00,2006-10-16 00:00:00,41,9
3,AR-4714,2002-12-18 00:00:00,2007-09-12 00:00:00,37,4
4,AR-4739,2002-12-18 00:00:00,2008-06-12 00:00:00,46,4
...,...,...,...,...,...
203,TL-09C,2005-05-10 15:45:00,2005-11-26 12:10:00,10,0
204,TL-14,2005-05-10 16:40:00,2005-11-08 15:45:00,7,0
205,TL-34C,2005-05-11 12:30:00,2005-11-26 14:10:00,10,0
206,TL-35C,2005-02-06 00:00:00,2024-01-30 00:00:00,208,18


EXPORTAR EXCEL EN RUTA ESPECIFICA

In [50]:
resultado.to_excel(r'G:\Unidades compartidas\03. Proyectos\20. FLOW\FLOW-003 Sistema Gestión de la Información\10_Colaborativos\01_Solicitudes\CODELCO-Dist. Norte\Modelo_Transporte_DRT\Base_de_datos_DRT_Mod_Trans_26062024\Temporalidad-QAQC.xlsx', sheet_name='Temporalidad', index=False)

REPORTE INFRAESTRUCTURAS 

In [1]:
import pyodbc
import pandas as pd


db_path = r"C:\Users\fdona\Downloads\BD_Hidrogeologia_Niveles_MLP_10062024.accdb"


conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    rf'DBQ={db_path};'
)


conn = pyodbc.connect(conn_str)

# Lee las tablas en DataFrames de pandas
def read_table_to_df(table_name):
    query = f'SELECT * FROM {table_name}'
    return pd.read_sql(query, conn)

# Especifica el nombre de la tabla que deseas leer
table_name = 'public_hg_mlp_infraestructuras_vw'
df = read_table_to_df(table_name)


df.columns





C:\Users\fdona\AppData\Local\Temp\ipykernel_22884\3783051630.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


Index(['OBJECT_ID', 'ID_PUNTO_HIDRO', 'ID_POZO', 'ESTE_WGS84', 'NORTE_WGS84',
       'Latitud', 'Longitud', 'ELEVACION MSNM', 'Sector',
       'Detalle Habilitación', 'Esquema Habilitacion', 'Pruebas hidraulicas',
       'Litologia'],
      dtype='object')

EXTRAER COLUMNAS NECESARIAS

In [2]:
extract_columns = ['ID_PUNTO_HIDRO','ESTE_WGS84', 'NORTE_WGS84',
       'ELEVACION MSNM']

# extraer columnas necesarias
df = df[extract_columns]

RENOMBRAR COLUMNAS PARA LOGRAR LA EQUIVALENCIA

In [3]:
rename_columns = ['PuntoMonitoreo','CoordenadaUTMEste', 'CoordenadaUTMNorte',
       'Elevacion_MSNM']

df.columns = rename_columns

In [5]:
df.to_excel(r'G:\Unidades compartidas\03. Proyectos\20. FLOW\FLOW-003 Sistema Gestión de la Información\10_Colaborativos\00_Proyectos\MLP\Anexos_Niveles-MLP\Infraestructuras_MLP.xlsx', sheet_name='Infraestructuras', index=False)

In [ ]:
# eliminar columnas de df fuente y source
df = df.drop(columns=['Fuente', 'Sourse'])

In [46]:
# df eliminar duplicados
df = df.drop_duplicates()

In [68]:
import pandas as pd

df['Fecha'] = pd.to_datetime(df['Fecha'])

# Agrupar por las columnas especificadas y calcular el promedio para las columnas deseadas
df_grouped = df.groupby(['ID_PUNTO_HIDRO', 'Sector', 'Fecha'], as_index=False).agg({
    'Nivel_msnm': 'mean',
    'Prof_mbnt': 'mean',
    'Prof_mbnr': 'mean'
})

print(df_grouped)

        ID_PUNTO_HIDRO            Sector      Fecha  Nivel_msnm  Prof_mbnt  \
0         ABF-10 (2AB)  Quillayes-Choapa 2007-01-02     987.050       1.95   
1         ABF-10 (2AB)  Quillayes-Choapa 2007-02-16     986.470       2.53   
2         ABF-10 (2AB)  Quillayes-Choapa 2007-03-20     986.000       3.00   
3         ABF-10 (2AB)  Quillayes-Choapa 2007-04-13     986.000       3.00   
4         ABF-10 (2AB)  Quillayes-Choapa 2007-05-30     985.850       3.15   
...                ...               ...        ...         ...        ...   
1041703     VictUgarte       Mauro-Pupío 2023-08-30     517.137      11.41   
1041704     VictUgarte       Mauro-Pupío 2023-09-13     517.147      11.40   
1041705     VictUgarte       Mauro-Pupío 2023-10-10     516.687      11.86   
1041706     VictUgarte       Mauro-Pupío 2023-11-08     517.067      11.48   
1041707     VictUgarte       Mauro-Pupío 2023-12-05     517.027      11.52   

         Prof_mbnr  
0              NaN  
1              NaN  


In [69]:
df = df_grouped

In [70]:
# Utiliza melt para transformar el DataFrame
df_melted = df.melt(
    id_vars=['ID_PUNTO_HIDRO', 'Fecha'],
    value_vars=['Nivel_msnm', 'Prof_mbnt', 'Prof_mbnr'],
    var_name='Parametro',
    value_name='Valor'
)



# Crea un diccionario para mapear los nombres de las columnas a las descripciones
parametro_mapping = {
    'Nivel_msnm': 'Cota del agua subterránea',
    'Prof_mbnt': 'Profundidad del agua subterránea bajo el nivel de terreno',
    'Prof_mbnr': 'Profundidad del agua subterránea bajo el punto de referencia'
}


df_melted['Parametro'] = df_melted['Parametro'].map(parametro_mapping)


unidad_medida_mapping = {
    'Cota del agua subterránea': 'msnm',
    'Profundidad del agua subterránea bajo el nivel de terreno': 'mbnt',
    'Profundidad del agua subterránea bajo el punto de referencia': 'mbpr'
}

df_melted['UnidadMedida'] = df_melted['Parametro'].map(unidad_medida_mapping)

df_melted.rename(columns={'ID_PUNTO_HIDRO': 'PuntoMonitoreo'}, inplace=True)

df_complete = df_melted[['PuntoMonitoreo', 'Parametro', 'Valor', 'Fecha', 'UnidadMedida']]


parametro_order = [
    'Profundidad del agua subterránea bajo el punto de referencia',
    'Profundidad del agua subterránea bajo el nivel de terreno',
    'Cota del agua subterránea',
]

df_complete['Parametro'] = pd.Categorical(df_complete['Parametro'], categories=parametro_order, ordered=True)

# Ordena el DataFrame por PuntoMonitoreo, Fecha y el orden específico de Parametro
df_complete = df_complete.sort_values(by=['PuntoMonitoreo', 'Fecha', 'Parametro'])

# Imprime el DataFrame resultante
df_complete

,PuntoMonitoreo,Parametro,Valor,Fecha,UnidadMedida
2083416,ABF-10 (2AB),Profundidad del agua subterránea bajo el punto...,NaN,2007-01-02,mbpr
1041708,ABF-10 (2AB),Profundidad del agua subterránea bajo el nivel...,1.950,2007-01-02,mbnt
0,ABF-10 (2AB),Cota del agua subterránea,987.050,2007-01-02,msnm
2083417,ABF-10 (2AB),Profundidad del agua subterránea bajo el punto...,NaN,2007-02-16,mbpr
1041709,ABF-10 (2AB),Profundidad del agua subterránea bajo el nivel...,2.530,2007-02-16,mbnt
...,...,...,...,...,...
2083414,VictUgarte,Profundidad del agua subterránea bajo el nivel...,11.480,2023-11-08,mbnt
1041706,VictUgarte,Cota del agua subterránea,517.067,2023-11-08,msnm
3125123,VictUgarte,Profundidad del agua subterránea bajo el punto...,NaN,2023-12-05,mbpr
2083415,VictUgarte,Profundidad del agua subterránea bajo el nivel...,11.520,2023-12-05,mbnt


In [59]:
df_complete = df_complete.drop_duplicates()

In [60]:
df_complete

,PuntoMonitoreo,Parametro,Valor,Fecha,UnidadMedida
6450319,ABF-10 (2AB),Profundidad del agua subterránea bajo el punto...,NaN,2007-01-02,mbpr
3932983,ABF-10 (2AB),Profundidad del agua subterránea bajo el nivel...,1.950,2007-01-02,mbnt
1415647,ABF-10 (2AB),Cota del agua subterránea,987.050,2007-01-02,msnm
6450320,ABF-10 (2AB),Profundidad del agua subterránea bajo el punto...,NaN,2007-02-16,mbpr
3932984,ABF-10 (2AB),Profundidad del agua subterránea bajo el nivel...,2.530,2007-02-16,mbnt
...,...,...,...,...,...
3206820,VictUgarte,Profundidad del agua subterránea bajo el nivel...,11.480,2023-11-08,mbnt
689484,VictUgarte,Cota del agua subterránea,517.067,2023-11-08,msnm
5724157,VictUgarte,Profundidad del agua subterránea bajo el punto...,NaN,2023-12-05,mbpr
3206821,VictUgarte,Profundidad del agua subterránea bajo el nivel...,11.520,2023-12-05,mbnt


In [71]:
n = 4
segment_size = len(df_complete) // n

# Divide el DataFrame en cinco partes
dataframes = [df_complete[i*segment_size:(i+1)*segment_size] for i in range(n-1)]
dataframes.append(df_complete[(n-1)*segment_size:]) 

# Exporta a un archivo Excel con cinco hojas
with pd.ExcelWriter('output.xlsx') as writer:
    for i, df_part in enumerate(dataframes):
        df_part.to_excel(writer, sheet_name=f'Parte_{i+1}', index=False)

# Verifica las dimensiones de cada DataFrame segmentado
for i, df_part in enumerate(dataframes):
    print(f'Parte {i+1}: {df_part.shape}')

Parte 1: (781281, 5)
Parte 2: (781281, 5)
Parte 3: (781281, 5)
Parte 4: (781281, 5)


In [ ]:
import pandas as pd

def partition_dataframe(df, n):
    """Divide un DataFrame en n partes casi iguales."""
    k, m = divmod(len(df), n)
    return (df[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))


# Divide el DataFrame en cinco partes
partes = list(partition_dataframe(df_complete, 8))

# Exporta cada parte a un archivo Excel diferente
for i, df_part in enumerate(partes):
    file_name = f'output_part_{i+1}.xlsx'
    df_part.to_excel(file_name, index=False)
    print(f'Parte {i+1} exportada a {file_name}')

# Verifica las dimensiones de cada DataFrame segmentado
for i, df_part in enumerate(partes):
    print(f'Parte {i+1}: {df_part.shape}')

In [47]:
import pandas as pd
from itertools import product

# Utiliza melt para transformar el DataFrame
df_melted = df.melt(
    id_vars=['ID_PUNTO_HIDRO', 'Fecha'],
    value_vars=['Nivel_msnm', 'Prof_mbnt', 'Prof_mbnr'],
    var_name='Parametro',
    value_name='Valor'
)

# Crea un DataFrame con todas las combinaciones posibles de ID_PUNTO_HIDRO, Fecha y Parametro
puntos_monitoreo = df['ID_PUNTO_HIDRO'].unique()
fechas = df['Fecha'].unique()
parametros = ['Nivel_msnm', 'Prof_mbnt', 'Prof_mbnr']

combinaciones = pd.DataFrame(list(product(puntos_monitoreo, fechas, parametros)), 
                             columns=['ID_PUNTO_HIDRO', 'Fecha', 'Parametro'])

# Combina el DataFrame de combinaciones con el DataFrame derretido
df_complete = pd.merge(combinaciones, df_melted, 
                       on=['ID_PUNTO_HIDRO', 'Fecha', 'Parametro'], 
                       how='left')

# Crea un diccionario para mapear los nombres de las columnas a las descripciones
parametro_mapping = {
    'Nivel_msnm': 'Cota del agua subterránea',
    'Prof_mbnt': 'Profundidad del agua subterránea bajo el nivel de terreno',
    'Prof_mbnr': 'Profundidad del agua subterránea bajo el punto de referencia'
}

# Aplica el mapeo a la columna 'Parametro'
df_complete['Parametro'] = df_complete['Parametro'].map(parametro_mapping)

# Añade la columna 'UnidadMedida' basada en el tipo de parámetro
unidad_medida_mapping = {
    'Cota del agua subterránea': 'msnm',
    'Profundidad del agua subterránea bajo el nivel de terreno': 'mbnt',
    'Profundidad del agua subterránea bajo el punto de referencia': 'mbpr'
}

df_complete['UnidadMedida'] = df_complete['Parametro'].map(unidad_medida_mapping)

# Renombra las columnas según el formato deseado
df_complete.rename(columns={'ID_PUNTO_HIDRO': 'PuntoMonitoreo'}, inplace=True)

# Agrupar y promediar los datos por PuntoMonitoreo, Fecha, y Parametro
df_grouped = df_complete.groupby(['PuntoMonitoreo', 'Fecha', 'Parametro', 'UnidadMedida'], as_index=False).agg({'Valor': 'mean'})

# Define el orden deseado de los parámetros
parametro_order = [
    'Cota del agua subterránea',
    'Profundidad del agua subterránea bajo el nivel de terreno',
    'Profundidad del agua subterránea bajo el punto de referencia'
]

# Convierte la columna 'Parametro' en una categoría con un orden específico
df_grouped['Parametro'] = pd.Categorical(df_grouped['Parametro'], categories=parametro_order, ordered=True)

# Ordena el DataFrame por PuntoMonitoreo, Fecha y el orden específico de Parametro
df_grouped = df_grouped.sort_values(by=['PuntoMonitoreo', 'Fecha', 'Parametro'])


df_grouped



,PuntoMonitoreo,Fecha,Parametro,UnidadMedida,Valor
0,ABF-10 (2AB),1970-03-17 00:00:00,Cota del agua subterránea,msnm,NaN
1,ABF-10 (2AB),1970-03-17 00:00:00,Profundidad del agua subterránea bajo el nivel...,mbnt,NaN
2,ABF-10 (2AB),1970-03-17 00:00:00,Profundidad del agua subterránea bajo el punto...,mbpr,NaN
3,ABF-10 (2AB),1970-04-14 00:00:00,Cota del agua subterránea,msnm,NaN
4,ABF-10 (2AB),1970-04-14 00:00:00,Profundidad del agua subterránea bajo el nivel...,mbnt,NaN
...,...,...,...,...,...
175041589,VictUgarte,2024-04-17 13:24:45,Profundidad del agua subterránea bajo el nivel...,mbnt,NaN
175041590,VictUgarte,2024-04-17 13:24:45,Profundidad del agua subterránea bajo el punto...,mbpr,NaN
175041591,VictUgarte,2024-04-24 00:00:00,Cota del agua subterránea,msnm,NaN
175041592,VictUgarte,2024-04-24 00:00:00,Profundidad del agua subterránea bajo el nivel...,mbnt,NaN
